In [1]:
%cd ..

/home/dawood/Workspace/compoundGAN


In [2]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader

from rdkit import Chem
from rdkit.Chem import Draw

from data.zinc import ZINC250K, ZINC_collate, Library
from data.qm9 import QM9, QM9_collate
from data.selfies import SELFIES

Dataset = QM9
collate = QM9_collate

from models.gcn import GCN

from tqdm import tqdm
import networkx as nx

from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
atom_list = [i for i in Library.atom_list]
atom_list.append('END')

In [4]:
model = GCN(58, 11, 256, [64, 128, 128])

In [5]:
dataset = SELFIES('~/Datasets/ZINC250K/data.csv')
#dataset = Dataset('/home/dawood/Datasets/QM9/data.txt')

#dataset.seq_length = 4

In [30]:
dataset[0].shape

[C]
[n]
[c]
[c]
[n]
[c]
[Ring1]
[Ring2]
[S]
[c]
[c]
[c]
[c]
[Branch1_3]
[epsilon]
[N]
[c]
[Branch1_3]
[epsilon]
[N]
[n]
[Ring1]
[Branch1_3]


torch.Size([23, 93])

In [29]:
max([dataset[0], dataset[1]], key=len).shape[0]

[C]
[n]
[c]
[c]
[n]
[c]
[Ring1]
[Ring2]
[S]
[c]
[c]
[c]
[c]
[Branch1_3]
[epsilon]
[N]
[c]
[Branch1_3]
[epsilon]
[N]
[n]
[Ring1]
[Branch1_3]
[C]
[S]
[C@@Hexpl]
[C]
[C]
[C@@Hexpl]
[Branch2_3]
[epsilon]
[Branch1_2]
[N]
[C]
[Branch1_3]
[epsilon]
[=O]
[N]
[Branch1_3]
[epsilon]
[C]
[C]
[c]
[c]
[c]
[c]
[c]
[c]
[c]
[c]
[c]
[c]
[Ring1]
[Branch2_2]
[Ring1]
[Branch1_1]
[C]
[Ring1]
[S]


36

In [ ]:
s = dataset.data[77590]
s = s.replace('][', '],[').split(',')

# s = [list(dataset.vocab).index(i) for i in s]
# s = torch.Tensor(s).long()
# s = F.one_hot(s, len(dataset.vocab))
s

In [ ]:
'\\' in '\\s'

In [ ]:
s

In [ ]:
dataset[0]

In [ ]:
dataloader = DataLoader(dataset, 2, shuffle=True, collate_fn=collate)

In [ ]:
for i, (g, atom_id, atom_x, atom_y, bond_y, logP) in tqdm(enumerate(dataloader)):
    break

In [ ]:
atom_x[:3, 1]

In [ ]:
atom_y[:, 1]

In [ ]:
def run(atom_y, bond_y):
    
    #print(G.ndata['feats'][:, :43])
    #print(G.edata['feats'][:, :5])
    atoms = [atom_list[i] for i in atom_y[:, 0, 0]]
    # atoms = [atom_list[i] for i in G.ndata['feats'][:, :43].argmax(1).data.cpu()]
    # aromatic = [i.item() for i in G.ndata['feats'][:, -2:].argmax(1).data.cpu()]

    G_pred = nx.Graph()
    num_nodes = len(atoms)
    aid = 0
    cid = 0
    for i, atom in enumerate(atoms):
        G_pred.add_node(i, atom='%s'%(atom), atom_id='%d'%(aid), compound_id=cid)
        aid += 1

    edges = []
    edge_types = []
    bn = 0
    for i in range(len(atoms)):
        for j in range(i):
            if (i > 12) and (j < (i - 12)):
                continue
            _j = j - (i - 12) if (i - 12) > 0 else j
            if bond_y[bn, 0, 0] > 0:
                edges.append((i, j))
                edge_types.append(bond_y[bn, 0, 0].item())
            bn += 1
                
    # edge_pred = G.edata['feats'][:, :5].argmax(1).data.cpu()
    # edge_pred_score = G.edata['feats'][:, :5].max(1)[0].data.cpu()
    for i in range(len(edges)):
        G_pred.add_edge(*edges[i], bond_type=edge_types[i])
    
#     if atoms[-1] == 'END' and len(atoms) > 1:
#         G_pred.remove_node(len(atoms) - 1)

    pos = nx.drawing.nx_agraph.graphviz_layout(G_pred)
    #pos = nx.kamada_kawai_layout(G_pred)
    node_labels = nx.get_node_attributes(G_pred, 'atom')
    edge_labels = nx.get_edge_attributes(G_pred, 'bond_type')
    cids = nx.get_node_attributes(G_pred, 'compound_id')
    edge_color_map = {
        0: 'm',
        1: 'b',
        2: 'r',
        3: 'g',
        4: 'y'
    }
    
    edge_colors = [edge_color_map[v] for k,v in edge_labels.items()]
    node_colors = [v for k, v in cids.items()]
    nx.draw_networkx(G_pred, pos, font_color='w', font_size=10, font_weight='bold',
                     node_size=256, labels=node_labels, #node_color=node_colors,
                     edge_color=edge_colors, width=2, cmap=plt.cm.Set1)
    
    node_labels = nx.get_node_attributes(G_pred, 'atom_id')
    for k in node_labels:
        node_labels[k] = '%10s'%node_labels[k]
    new_pos = {}
    for k, (x, y) in pos.items():
        new_pos[k] = (x, y)
    nx.draw_networkx_labels(G_pred, new_pos, labels=node_labels, font_size=10, font_weight='bold',)

In [ ]:
run(atom_y, bond_y)

In [ ]:
mol = Chem.MolFromSmiles('NC(=O)C1(CO1)C(N)=O')
Draw.MolToMPL(mol); plt.show()


In [ ]:
atom_y.shape

In [ ]:
model(g).shape

In [ ]:
x = torch.Tensor([10, 3]).cuda()

In [ ]:
y = torch.Tensor([5])
y = y.to(x)
y

In [ ]:
bond_y.shape

In [ ]:
mask.shape

In [ ]:
g

In [ ]:
a = list(range(15))

In [ ]:
a[:0:-1][:12]

In [ ]:
a[::-1]